# Documentation Code Example from:
- http://spark.apache.org/docs/latest/streaming-programming-guide.html#a-quick-example

## A Quick Example

Before we go into the details of how to write your own Spark Streaming program, let’s take a quick look at what a simple Spark Streaming program looks like. Let’s say we want to count the number of words in text data received from a data server listening on a TCP socket. All you need to do is as follows.

In [1]:
import findspark

In [2]:
findspark.init( '/home/kajili/spark-2.4.5-bin-hadoop2.7' )

First, we import StreamingContext, which is the main entry point for all streaming functionality. We create a local StreamingContext with two execution threads, and batch interval of 1 second.

In [3]:
from pyspark import SparkContext

In [4]:
from pyspark.streaming import StreamingContext

In [5]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)

Using this context, we can create a DStream that represents streaming data from a TCP source, specified as hostname (e.g. localhost) and port (e.g. 9999).

In [6]:
# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream('localhost',9999)

This lines DStream represents the stream of data that will be received from the data server. Each record in this DStream is a line of text. Next, we want to split the lines by space into words.

In [7]:
# Split each line into words
words = lines.flatMap(lambda line: line.split(' '))

flatMap is a one-to-many DStream operation that creates a new DStream by generating multiple new records from each record in the source DStream. In this case, each line will be split into multiple words and the stream of words is represented as the words DStream. Next, we want to count these words.

In [8]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

The words DStream is further mapped (one-to-one transformation) to a DStream of (word, 1) pairs, which is then reduced to get the frequency of words in each batch of data. Finally, wordCounts.pprint() will print a few of the counts generated every second.

Note that when these lines are executed, Spark Streaming only sets up the computation it will perform when it is started, and no real processing has started yet. To start the processing after all the transformations have been setup, we finally call

In [9]:
ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2020-02-24 13:00:41
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:42
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:43
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:44
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:45
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:46
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:47
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:48
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:00:49
----------

-------------------------------------------
Time: 2020-02-24 13:01:52
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:01:53
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:01:54
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:01:55
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:01:56
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:01:57
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:01:58
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:01:59
-------------------------------------------

-------------------------------------------
Time: 2020-02-24 13:02:00
----------

KeyboardInterrupt: 

The complete code can be found in the Spark Streaming example NetworkWordCount.

If you have already downloaded and built Spark, you can run this example as follows. You will first need to run Netcat (a small utility found in most Unix-like systems) as a data server by using

`nc -lk 9999`